In [1]:
import numpy as np
from pylibdl.data import DataLoader
from pylibdl.optim import Adam
from utils import DistractedDriver, MyResNet, validate
from pylibdl import cross_entropy_with_logits
import time
np.random.seed(0)

In [2]:
path = '/home/superbabes/Downloads/ddriver'
batch_size = 64
lr = 1e-4
log_every = 10  # 10

In [3]:
train_data = DistractedDriver(path, val=False)
train_loader = DataLoader(train_data, batch_size, shuffle=True, drop_last=True)
val_data = DistractedDriver(path, val=True)

In [4]:
model = MyResNet()
optim = Adam(model.parameter(), lr)

In [5]:
begin = time.time()
print('iteration | loss')
for i, (imgs, labels) in enumerate(train_loader):
    pred = model(imgs)
    loss = cross_entropy_with_logits(pred, labels)
    loss.backward()
    optim.step()
    optim.zero_grad()
    if i % log_every == 0:
        print(f'{i: <9} | {np.array(loss.data):.4f}')
    if i >= 100:
        break
model.save('model')
print(f'validation accuracy: {validate(model, val_data)*100:.2f}%')
print(f'training took {time.time() - begin: .2f} seconds')

iteration | loss
0         | 2.3026
10        | 2.0798
20        | 1.7576
30        | 1.5485
40        | 1.4815
50        | 1.6666
60        | 1.6329
70        | 1.4992
80        | 1.4835
90        | 1.4169
100       | 1.4509
validation accuracy: 58.36%
training took  256.37 seconds


In [ ]:
#training took  243.76 seconds
#training took  231.65 seconds